In [0]:
!pip3 install python-chess==0.30.0
!pip install -q pyyaml h5py
import chess
import numpy as np
import math
import random
import copy
import chess.svg
import tensorflow as tf
from tensorflow import keras
from IPython.display import SVG, display
from IPython.display import clear_output

     |████████████████████████████████| 133kB 2.8MB/s 
  Found existing installation: python-chess 0.23.11
    Uninstalling python-chess-0.23.11:
      Successfully uninstalled python-chess-0.23.11


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
class Node:
    def __init__(self,boardState,turn,parent):
        self.boardState=boardState
        self.totalVal=0
        self.visitCount=0
        self.children=[]
        self.parent=parent
        self.fullyExpanded=False
        self.turn=turn

In [0]:
    model = keras.Sequential([
    keras.Input(shape=(769,)),
    keras.layers.Dense(units=512, activation='sigmoid'),
    keras.layers.Dense(units=512, activation='sigmoid'),
    keras.layers.Dense(units=128, activation='sigmoid'),
    keras.layers.Dense(units=128, activation='sigmoid'),
    keras.layers.Dense(units=32, activation='sigmoid'),
    keras.layers.Dense(units=32, activation='sigmoid'),
    keras.layers.Dense(units=16, activation='sigmoid'),
    keras.layers.Dense(units=1, activation='sigmoid')
])
    
    model.compile(optimizer='SGD',
              loss='mean_squared_error',
              metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
def getBoardState(board):
  board_state=np.zeros(769)
  for i in range(64):
    if(board.piece_type_at(i) is not None):
      if(board.color_at(i) is True):
        board_state[(board.piece_type_at(i)-1)*64 + i ] = 1
      else:
        board_state[(12 - board.piece_type_at(i))*64 + i ] = 1
  if board.turn :
      board_state[768]=10
  else :
      board_state[768]=-10
  return board_state

In [0]:
def getEval(board):
  board_state=getBoardState(board)
  input_state=board_state.reshape(1,769)
  prediction=model.predict(input_state)
  return (np.sum(prediction) - 0.5)*10

In [0]:
def MCTS(root): 
    for i in range(150): 
        leaf = traverse(root)
        #print("leaf state after",i, "iters:")
        #print(leaf.boardState)
        rollout_result = rollout(leaf)
        #print("result after",i,"iter:",rollout_result)
        backpropogate_results(root,leaf,rollout_result)
    max_visits=-1
    best_child_num=0
    child_num=0
    visits=[child.visitCount for child in root.children]
    maxpos = visits.index(max(visits))
    best_state=root.children[maxpos].boardState
    printVal=max(visits)*len(list(root.boardState.legal_moves))/150.0
    print(printVal)
#    for child in root.children:
        #print(child.boardState)
        #print(child.visitCount)
        #print(child.totalVal)
#        if child.visitCount>max_visits:
#            max_visits=child.visitCount
#            best_child_num=child_num
#            best_state=copy.deepcopy(child.boardState)
#        child_num=child_num+1
    #print(root.children[best_child_num].boardState)
    return best_state

In [0]:
def backpropogate_results(root,leaf,result):
    while leaf is not None:
        leaf.totalVal=leaf.totalVal + result*-1*leaf.turn
        leaf.visitCount=leaf.visitCount+1
        leaf=leaf.parent

In [0]:
def rollout(node):
    evaluation=0
    res=node.boardState.result()
    if (node.boardState.is_game_over()):
      if(res == "1-0"):
        evaluation=10
      elif(res == "0-1"):
        evaluation=-10
    else:
      evaluation=getEval(node.boardState)
    return evaluation

In [0]:
def makeRandomAction(board,turn):
    allMoves=list(board.legal_moves)
    board.push(allMoves[random.randint(0,len(allMoves)-1)])
    #print(nextBoards)
    return board

In [0]:
def traverse(node): 
    while node.fullyExpanded and not((node.boardState).is_game_over()): 
        node=best_uct(node)
    if node.visitCount==0: 
        return node
    else: 
        node=fullyExpand(node)
        return randomChild(node)

In [0]:
def best_uct(node):
    expConst=10
    bestNode=None
    bestVal=-10000
    for child in node.children:
        uctVal=(child.totalVal/(child.visitCount+0.01)) + expConst * math.sqrt(node.visitCount)/(child.visitCount + 1) 
        if uctVal>bestVal:
            bestVal=uctVal
            bestNode=child
    return bestNode

In [0]:
def fullyExpand(node):
    node.fullyExpanded=True
    if (node.boardState).is_game_over():
        return node
    else:
        allMoves=list((node.boardState).legal_moves)
        for move in allMoves:
          (node.boardState).push(move)
          node.children.append(Node(copy.deepcopy(node.boardState),-node.turn,node))
          (node.boardState).pop()
    return node

In [0]:
def randomChild(node):
    
    if len(node.children)==0:
        return node
    else:
        return node.children[random.randint(0,len(node.children)-1)]

In [0]:
model=tf.keras.models.load_model('model_final.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
w=0
l=0
t=0
for k in range(101):
  print(k)
  player=1
  board = chess.Board()  
  board_state=getBoardState(board)
  train_states=board_state
  moves=0
  while not(board.is_game_over()):
      if player is 1:
          if(random.random()<0.05 or (moves<17 and random.random()<0.6)):
            board=makeRandomAction(board,player)
          else:
            board=MCTS(Node(board,player,None))
          board_state=getBoardState(board)
          train_states=np.vstack((train_states,board_state))
          player=-1
      else:
          if(random.random()<0.05 or (moves<17 and random.random()<0.6)):
            board=makeRandomAction(board,player)
          else:
            board=MCTS(Node(board,player,None))
          board_state=getBoardState(board)
          train_states=np.vstack((train_states,board_state))
          player=1
      moves=moves+1
  if(board.result() == "1-0"):
    train_results=np.ones(len(train_states))
    w=w+1
  elif(board.result() == "0-1"):
    train_results=np.zeros(len(train_states))
    l=l+1
  else:
    train_results=(np.ones(len(train_states)))*0.5
    t=t+1
  print("white:",w)
  print("black:",l)
  print("draws:",t)

  combined = list(zip(train_states, train_results))
  random.shuffle(combined)
  train_states[:], train_results[:]=zip(*combined)
  model.fit(train_states, train_results, epochs=10)    
  if(k%10==0):
    model.save('model_final.h5')

Streaming output truncated to the last 5000 lines.
1.0
1.04
1.02
1.12
1.0
1.04
1.02
1.12
1.0
1.04
1.02
1.12
1.0
1.04
1.02
1.12
1.0133333333333334
1.02
1.04
1.0133333333333334
1.0
1.0266666666666666
1.0733333333333333
1.0
1.0733333333333333
1.02
1.0
1.0
1.0733333333333333
1.02
1.0
1.0
1.0733333333333333
1.02
1.0
1.0
1.0733333333333333
1.02
1.0
1.0
1.04
1.02
1.0
1.0
1.04
1.02
1.0
1.0
1.04
1.02
1.0
1.0
1.04
1.02
1.0
1.0
1.12
1.02
1.04
1.0
1.0
1.02
1.04
1.3
white: 18
black: 36
draws: 36
Train on 243 samples
Epoch 1/10
243/243 [==============================] - 0s 127us/sample - loss: 0.1009 - acc: 0.0000e+00
Epoch 2/10
243/243 [==============================] - 0s 91us/sample - loss: 0.0963 - acc: 0.0000e+00
Epoch 3/10
243/243 [==============================] - 0s 85us/sample - loss: 0.0916 - acc: 0.0000e+00
Epoch 4/10
243/243 [==============================] - 0s 93us/sample - loss: 0.0870 - acc: 0.0000e+00
Epoch 5/10
243/243 [==============================] - 0s 110us/sample - loss: 0.08

In [0]:
player=1
board = chess.Board()  
board_state=getBoardState(board)
train_states=board_state
while not(board.is_game_over()):
    if player is 1:
        board=MCTS(Node(board,player,None))
        board_state=getBoardState(board)
        train_states=np.vstack((train_states,board_state))
        player=-1
    else:
        a=input()
        playerMove=board.parse_san(a)
        board.push(playerMove)
        board_state=getBoardState(board)
        train_states=np.vstack((train_states,board_state))
        player=1
    print(board)
    print("\n")

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R




KeyboardInterrupt: ignored

In [0]:
for k in range(2):
  print(k)
  player=1
  board = chess.Board()  
  board_state=getBoardState(board)
  train_states=board_state
  while not(board.is_game_over()):
      if player is 1:
          if(random.random()<0.05):
            board=makeRandomAction(board,player)
          else:
            board=MCTS(Node(board,player,None))
          board_state=getBoardState(board)
          train_states=np.vstack((train_states,board_state))
          player=-1
      else:
          if(random.random()<0.05):
            board=makeRandomAction(board,player)
          else:
            board=MCTS(Node(board,player,None))
          board_state=getBoardState(board)
          train_states=np.vstack((train_states,board_state))
          player=1
      print(board)
      print("\n")
  if(board.result() == "1-0"):
    train_results=np.ones(len(train_states))
  elif(board.result() == "0-1"):
    train_results=np.zeros(len(train_states))
  else:
    train_results=(np.ones(len(train_states)))*0.5

  combined = list(zip(train_states, train_results))
  random.shuffle(combined)
  train_states[:], train_results[:]=zip(*combined)
  model.fit(train_states, train_results, epochs=10)    
  if(True):
    model.save('modeltry2.h5')

0
8
11.495790481567383


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . P .
P P P P P P . P
R N B Q K B N R


r n b q k b n r
p p p p p . p p
. . . . . . . .
. . . . . p . .
. . . . . . . .
. . . . . . P .
P P P P P P . P
R N B Q K B N R


8
11.495954990386963


r n b q k b n r
p p p p p . p p
. . . . . . . .
. . . . . p . .
. . . . . . . .
. . . P . . P .
P P P . P P . P
R N B Q K B N R


8
-11.49695634841919


r n b q . b n r
p p p p p k p p
. . . . . . . .
. . . . . p . .
. . . . . . . .
. . . P . . P .
P P P . P P . P
R N B Q K B N R


6
8.62206220626831


r n b q . b n r
p p p p p k p p
. . . . . . . .
. . . . . p . .
. . . . . . . .
. . . P . . P B
P P P . P P . P
R N B Q K . N R


7
-10.059775114059448


r n b . q b n r
p p p p p k p p
. . . . . . . .
. . . . . p . .
. . . . . . . .
. . . P . . P B
P P P . P P . P
R N B Q K . N R


6
8.622005581855774


r n b . q b n r
p p p p p k p p
. . . . . . . .
. . . . . p . .
. . . . . . . .
. 